In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# The Cthulhu generation

In [2]:
import numpy as np

text_name = 'The Call of Cthulhu.txt'
text = open(text_name).read().lower()
len(text)

69459

In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 23133
Unique characters: 51
Vectorization...


In [4]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               92160     
_________________________________________________________________
dense_1 (Dense)              (None, 51)                6579      
Total params: 98,739
Trainable params: 98,739
Non-trainable params: 0
_________________________________________________________________


In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
import random
import sys

Epochs = 100

for epoch in range(1, Epochs+1):
    print('Epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 300 characters
        for i in range(300):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch 1
Epoch 1/1
23133/23133 [==============================] - 21s 887us/step - loss: 3.0044
--- Generating with seed: "he two sounds frequently repeated are those rendered by the "
------ temperature: 0.2
he two sounds frequently repeated are those rendered by the  e   e  t e        i  s  e   e i      e    oe e  e e      e  d o   e       e e   e  e  e    ae o e    er   e    es  s     e    t  e o s  s    o e  e  i es   o  e  e   e i e  e s e ir  e  s    o   e    e e  e      e       i e    e  o  i   i r  i e  e s  e  a te     e e  e e  e  ot   a  s    s   e ee 
------ temperature: 0.5
r  i e  e s  e  a te     e e  e e  e  ot   a  s    s   e ee o h inoe or  et oio i  ris t t a ehsec o  ait ce i utos ys e e es ta or o  ts a eas s h n s   one e e a tit  oe taeea  ine e i  i d iu c   te rn hi it s  sad ls mtcrte esia eeps s  oo  e ds ttn  e i rete t  h trh e sg terssi  aestn s io   ro ct i eahoh rrm  osos rs m tas e  r r e r  ao fy   nnea rd 
------ temperature: 1.0
o ct i eahoh rrm  osos

stound of the sont of the the was and and of the the some and the meathe stoms and incingsing of leam he the pound and the thane the prithing of hithe stome forstoug the sound the the soung hegrofe in chall farse the soreded amd store wath se foved at of an tha deathe the sthen thas of man of the and and and nosting and porestof the thay buthe stares of mesc
------ temperature: 1.0
d and and nosting and porestof the thay buthe stares of mescines rexisce4s, ind sowthes delllang it sinasod fof sousslancew bi
chesn und al and ameny cullalit: vewhaith. thes in loming gleongist thisgothicrals masl, ig inderond ffarchinkd woo gulapray. and ham wis ousbio bastomedea def aper, fom foulke toonmarding deatbe nowand thoughed an of frnop the weste
------ temperature: 1.2
lke toonmarding deatbe nowand thoughed an of frnop the weste, wnouncs vraono hargwolemlly ion. 
thimvink of c4us,lgrobed cpom? hed pnowvutemacll bancoke sroemosty kont'waw tvoc 1ss aatousd menk, ont meist them wild the lisint of a

h four cases of vague description suggest fugitive glimpses and the seresting of the store the store and the ment of the the store the seresting of the in the store the mast of the sound and the store the store the the sound the store the store the store the menter and the menting of the monesting the anter the monesting of the menter sound the store the rea
------ temperature: 0.5
he anter the monesting of the menter sound the store the reacines of the mase and his and could so couss of could hid on and the proitien the stint of the some the sound on the linge in the the sting ol dene pore and the stound why the the stses of the oreden soden whith souss berit of the salle and ace seat the sters on serass of thes and inse fan the pore 
------ temperature: 1.0
 ace seat the sters on serass of thes and inse fan the pore by was gah on ips.rrises st mapely out iesture stoun oe stest of sereptiteson of same stores ip or the uores, and rothinily bloughiges of the amiss aos. an, unviaisainty,

sting of the streation and the stren and the store the strem in the stersed a fripele of maly whing hinh of the moust his will bleat in whe the orith of the men all of the strames of the pratse of the mant seen and cote the ore pristed of hid his which and a cuntere and reatanter stone and of the suich of the with the surele and tien but the preacies in the 
------ temperature: 1.0
ich of the with the surele and tien but the preacies in the sotsed in pureite ay whats vicicies ristullipic bucking by a kund of thou ginget lost sompon ath great thirg dinter on batien th. it he westing cubter lenkint whocs ourd that timn its visl. hou, ne mears in blow cortern in and the - shas nothiss thit greavon, whiy-swniction preem not in stoussnal -e
------ temperature: 1.2
thiss thit greavon, whiy-swniction preem not in stoussnal -eable but4es mown byzuded the cralt, had al-,inggiag, under of lince wiadk; hus wtinare ol. it harch iges"incting mboum verb, a -wish faace unenker do in gryalll ryechlils

 evilly on a rectangular block or pedestal covered with under come the stone in the streation of the stone of the strone of the stone in the stone of the stone of the sting of the strone of the sting of the strante of the stone in the stone of the stone of the strante and the stone of the stren and the stone of the stone of the strone of the strones of the s
------ temperature: 0.5
the stone of the stone of the strone of the strones of the storming the stenting of corcal in shing to in and for the reeper and withtressere the wat seen forstioned hes ane and antares mont some of a the stone were the stone of the stem to and but the was the sione lind carted the was but cale could head had hear hes of the ston the portsone wish cult and i
------ temperature: 1.0
d head had hear hes of the ston the portsone wish cult and itll the. they thing her s meptroted, hid, if of manils ored apveriass beecledi ond thoug kenolest and nongiomy and. aguer hif seation at sudsiatse henderm. but tham queat

...

 the eleat of the stick of the sting the porthous which the could come and his found with stone and were stillud. they froutte to the resistor and corring and more marions wisk, buc thought was besough the be antage whener cigeere in sthat from than the endre professor and thing from that impricity, af instrong intone of the store withous from that in persou
------ temperature: 1.0
 af instrong intone of the store withous from that in persous but eavan aborayly y cugttion. ittinest cory. hult imbering, amough tret sariou hownhe tiones lons as hears, surcerwndf about the med even at in asture of eve them ac and him ols sigo pincors, derpering witas of in hach not mong with suspew darken in a alinians sooms which he reeligize in. oms and
------ temperature: 1.2
ew darken in a alinians sooms which he reeligize in. oms and clorred's the sushtral very condsisity formane it hav deep to dirrerastion oulb, car of thear, was the jastex mence boto, one ond could not vesured, biflers 12121, tinds

 feelers, a scaly, rubbery-looking body, prodigious claws on sing mans and the saight were striege wene the scult of the elden sing therelly grone of shell the legen lent its of the which there veseris, and the court and contion of the strange conting his manion of the ereat on the strange of the seakly dramm to the disture of it had had ever leacked he was 
------ temperature: 0.5
akly dramm to the disture of it had had ever leacked he was s in ercaller canteritl risustions of the dies mons sulle and lakn hourhed hid, dramplems and the mont of the mont of the squatt could this trongsicred, and he men and sighus washing and he sellict an a guatthe before of hushereat frothor an whilsc olden semilicall rectingss by the summ comen was bl
------ temperature: 1.0
n whilsc olden semilicall rectingss by the summ comen was bleceled with its not soline it his had know not me soun wert surch wale. in stord the nconnow was a stulter the clainsw'y gastre, bos spacand townstean whes the mourabed c

n ingiong died and the man angioning the scounterion the strante prosoners of the cenoued song the ming out and lite stall whils mege in he suld bo dnepris singulity of the mes ans store of an aghors and an angilys. in but on mows ancurctyon line ant i cult for the mestinged to sonesting it in the sod that expocsed and deraby and the dor beatust and hire and
------ temperature: 1.0
od that expocsed and deraby and the dor beatust and hire and trie of aly brally. the tores and tat of hux waich made be one itsoons benousterred to courcapson the spmight faerith the gadeed it of homsteras alys.. 
to gin tralianibefough hide swils cult new, and not the firilings formy lit, mans beinkeon leth of. the an sown of the man seen one therells quist
------ temperature: 1.2
keon leth of. the an sown of the man seen one therells quistries a cendul mes the oldrepssict afouluted in the curater sprocoon he veimstring wile intered weha intluct derode of the bospars, torrestion to mearrest, it were amone, 

 winter of 1926-27 with the death of my great-uncle, george allittitand towndredond lent of theur of that helurre hid, had there felle had had colden befooness of the scult of the eleat of the store of aptia sthese upor wera sicked be on. 
the water the professor angell of the which could not of the brack come had come priss. they of the elat of the men and 
------ temperature: 0.5
 brack come had come priss. they of the elat of the men and sought was ochioni and motion with from outs in shelp no san angulition which of aptis parch 1215, this now hid not mentical sint it insticatain butd, and from that the eldat with some of them caseres which of apter conmitton from tham thing from in and morion inaritity in respotsed thereighed a rep
------ temperature: 1.0
g from in and morion inaritity in respotsed thereighed a reptill's from than in and itwe anly pevilily lenell nowed the cult voyoratt wehe the froghede. 
he a hid han someppront ctollared inabation. the cultant of the which thene,

the an went browner uprisse of dams working the professor angell profaser wish his deam 
------ temperature: 1.0
of dams working the professor angell profaser wish his deam of at had haverysus. 
johan se pight squatteroh dears ablere, and quelll made of sope inslice somplece in oreecin mont corlding stong. the buttay which dreams. cheet men undelims insogble sight in the sound cititualle for imbit that in exiricin abrimity. they prisiens a arouf authom were faretiod; w
------ temperature: 1.2
icin abrimity. they prisiens a arouf authom were faretiod; whe e ererace3s; bat what had come the uritequiss, but from wilp crillad free dand the seldert; and abery nom to kin wastige anquiry from batant wordins recoolyen integnis eadh stoke. the rate fulfumd nor by dat faruting sove sleat lattly and hoquean he water farchu1s jehassere, dethat hear sund unal
Epoch 95
Epoch 1/1
23133/23133 [==============================] - 17s 744us/step - loss: 0.7088
--- Generating with seed: "elief on which the

In [7]:
model.save_weights('cthulhu_weights.h5')